### Functions for non-conserved species

In [1]:
def prep_lines(lines):
    for i in range(len(lines)):
        lines[i]=lines[i].replace('     * ', '')
        lines[i]=lines[i].replace(' \n', '')
        lines[i]=lines[i].replace('\n', '')
    return lines

'''
Read fortran ODEs and prep for translation.
    - file = file name
    - returns: dict
'''
def read_fortran_ODEs(file, cons = False):

    ## read fortran ODEs from given file
    with open(file, 'r') as f:
        lines = f.readlines()

    if cons == True:
        lines = lines[16:93]
    if cons == False:
        ## select non-conserved species
        lines = lines[123:]

    ## remove unnecessary fortran code
    lines = prep_lines(lines)

    ODEs = stick_fortran(lines)

    return ODEs

'''
make dict:
    - entry per spec
    - stick together separate fortran strings
'''
def stick_fortran(lines):
    
    ODEs = dict()

    for i, line in enumerate(lines):
        if len(line) != 0 and line[0] == 'C':
            ODEs[line[2:]] = str('')
            print('i=',i, end = '\r')
            start = i + 1

            for j in range(start,start+1000):
                if j >= len(lines):
                    break
                if len(lines[j]) != 0 and lines[j][0] == 'C':
                    stop = j
                    break
                
            for j in range(i+1,stop):
                ODEs[line[2:]] += lines[j]

            i = stop +1

    count = 0
    for key in ODEs:
        if key == '':
            count += 1

    while count > 0:
        del ODEs['']
        count -= 1

    return ODEs


'''
Translate fortran ODE to python ODE
'''
def fortran_to_python(ODEs, spec, cons = False):
    print(spec)

    if isinstance(ODEs[spec],str):
        ## NOT fractional abundances, but in units of cm^-3
        ODEs[spec] = ODEs[spec].replace('*HNR', '')
        ## split string
        ODEs[spec] = ODEs[spec].split(' ')

    ## remove empty strings
    to_remove = ''
    while to_remove in ODEs[spec]:
        ODEs[spec].remove('')

    ## remove other fortran commands
    while 'END' in ODEs[spec]:
        ODEs[spec].remove('END')
    while 'RETURN' in ODEs[spec]:
        ODEs[spec].remove('RETURN')

    # print('Conserved? ',cons)
    ## Translate
    if cons == True:
        # print('Conserved? ',cons)
        ## replace brackets: ( --> [
        print(len(ODEs[spec]))
        for i in range(len(ODEs[spec])):
            
            # if ODEs[spec][i-1] != '+' or ODEs[spec][i-1] != '-' or ODEs[spec][i-1] != '*' or ODEs[spec][i-1] != '/' or ODEs[spec][i-1] != ')' or ODEs[spec][i-1] != '(':
            ODEs[spec][i] = ODEs[spec][i].replace('(', '[')
            ODEs[spec][i] = ODEs[spec][i].replace(')', ']')
            ODEs[spec][i] = ODEs[spec][i].replace('[+', '(+')
            ODEs[spec][i] = ODEs[spec][i].replace(']]', '])')

    if cons == False:
        ## replace brackets: ( --> [
        for i in range(len(ODEs[spec])):
            if ODEs[spec][i][0] == 'F' or ODEs[spec][i][0] == 'D':
                ODEs[spec][i] = ODEs[spec][i].replace('(', '[')
                ODEs[spec][i] = ODEs[spec][i].replace(')', ']')

        ## replace final line
        last = ODEs[spec][-1]
        # print('last', last)
        while not last[0] == 'Y':
            ODEs[spec].remove(last)
            last = ODEs[spec][-1]
        # print('at the end',last)
        for i in range(5,len(last)):
            if last[i] == ')':
                stop = i
                break
        nb = last[5:stop]

        ODEs[spec][-1] = 'YDOT['+nb+']=F-(D*Y['+nb+'])'

    return ODEs
    

### Read in file & prep

In [2]:
file = '/lhome/silkem/CHEM/src-IP-AP-HNR/code/acodes.f'

with open(file, 'r') as f:
    lines = f.readlines()

ODEs = read_fortran_ODEs(file)


In [3]:
ODEs['C11+']

'      F=0.+K(1517)*Y(10)*Y(458)*HNR+K(1518)*Y(10)*Y(455)*HNR+K(2066)*Y(11)*Y(453)*HNR+K(2068)*Y(11)*Y(459)*HNR+K(5727)*Y(465)+K(6042)*Y(10)*Y(451)*HNR+K(6202)*Y(465)+K(6538)*Y(465)      D=0.+K(994)*X(1)*HNR+K(995)*X(1)*HNR+K(996)*X(1)*HNR+K(997)*X(1)*HNR      YDOT(466)=F-(D*Y(466))      RETURN      END'

### Translate

In [4]:
for key in ODEs:
    ODEs = fortran_to_python(ODEs, key)

H
H+
H-
H2+
H3+
He
He+
HeH+
C-
C+
C
CH-
CH+
CH
CH2+
N+
CH2
N
CH3
NH
NH+
CH3+
O+
NH2+
CH4+
O-
NH2
O
CH4
OH-
OH
OH+
CH5+
NH3+
NH3
NH4+
H2O
H2O+
F
H3O+
F+
HF
HF+
H2F+
Na
Na+
C2-
C2+
Mg
Mg+
C2
C2H-
C2H+
C2H
CN
C2H2
C2H2+
CN-
CN+
HCN
C2H3
C2H3+
HCN+
HNC
N2
C2H4+
Si+
H2CN
H2NC+
N2+
Si
CO
CO+
C2H4
HCNH+
C2H5+
HCO
SiH+
HCO+
CH2NH
SiH
C2H5
HOC+
N2H+
CH2NH2+
SiH2
CH3CH3+
NO
H2CO
CH4N+
H2CO+
CH3CH3
SiH2+
NO+
P+
P
H3CO+
HNO+
SiH3+
HNO
CF+
C2H7+
SiH3
O2
SiH4
PH
S
H2NO+
CH3OH
SiH4+
O2+
S-
O2-
PH+
CH3OH+
S+
CH3OH2+
O2H
PH2+
HS
HS+
PH2
O2H+
SiH5+
H2O2
H2S+
H2S
PH3+
H3S+
Cl
Cl+
C3+
HCl
C3-
C3
HCl+
H2Cl+
C3H-
C3H+
C3H
C2N
C3H2+
C2N+
H2CCC
CNC+
C3H2
C2NH+
C3H3+
CH2CCH
CH2CCH+
SiC
CH2CCH2
SiC+
C2O
C3H4+
C2O+
CH2CN
CH3CCH
CH2CN+
C3H5+
HCSi
HCSi+
CH3CN+
HC2O+
CH3CN
C3H6+
SiN
CH3CNH+
NH2CN
CH3CHCH2
SiN+
SiCH2+
SiCH2
CNO
CH2CO+
OCN
OCN+
CH2CO
CP+
HNCO
HCNO+
HNCO+
HONC
SiCH3+
HCNO
CP
C3H7+
NH2CNH+
HNSi+
HOCN
HNSi
CH3CO+
HOCN+
SiCH3
HONC+
CS+
HCNOH+
N2O+
CS
SiCH4+
CO2+
H2OCN+
HNCOH+
SiNH2+
HCP
SiO+
CH3CHO
SiO
H

In [5]:
ODEs['C11+']

['F=0.+K[1517]*Y[10]*Y[458]+K[1518]*Y[10]*Y[455]+K[2066]*Y[11]*Y[453]+K[2068]*Y[11]*Y[459]+K[5727]*Y[465]+K[6042]*Y[10]*Y[451]+K[6202]*Y[465]+K[6538]*Y[465]',
 'D=0.+K[994]*X[1]+K[995]*X[1]+K[996]*X[1]+K[997]*X[1]',
 'YDOT[466]=F-(D*Y[466])']

### Conserved species

In [6]:
ODEs_cons = read_fortran_ODEs(file, True)

In [7]:
ODEs_cons

{'e-': '      X(1)=0.+TOTAL(1)+(+Y(2)-Y(3)+Y(4)+Y(5)+Y(7)+Y(8)-Y(9)+Y(10)-Y(12)+Y(13)+Y(15)+Y(16)+Y(21)+Y(22)+Y(23)+Y(24)+Y(25)-Y(26)-Y(30)+Y(32)+Y(33)+Y(34)+Y(36)+Y(38)+Y(40)+Y(41)+Y(43)+Y(44)+Y(46)-Y(47)+Y(48)+Y(50)-Y(52)+Y(53)+Y(57)-Y(58)+Y(59)+Y(62)+Y(63)+Y(66)+Y(67)+Y(69)+Y(70)+Y(73)+Y(75)+Y(76)+Y(78)+Y(79)+Y(83)+Y(84)+Y(85)+Y(87)+Y(90)+Y(91)+Y(93)+Y(94)+Y(95)+Y(97)+Y(98)+Y(99)+Y(101)+Y(102)+Y(108)+Y(110)+Y(111)-Y(112)-Y(113)+Y(114)+Y(115)+Y(116)+Y(117)+Y(119)+Y(121)+Y(123)+Y(124)+Y(126)+Y(128)+Y(129)+Y(131)+Y(132)-Y(134)+Y(136)+Y(137)-Y(138)+Y(139)+Y(142)+Y(143)+Y(145)+Y(147)+Y(148)+Y(150)+Y(153)+Y(155)+Y(156)+Y(159)+Y(160)+Y(162)+Y(163)+Y(164)+Y(166)+Y(168)+Y(171)+Y(172)+Y(175)+Y(177)+Y(179)+Y(181)+Y(182)+Y(184)+Y(187)+Y(188)+Y(189)+Y(192)+Y(193)+Y(195)+Y(196)+Y(197)+Y(198)+Y(200)+Y(201)+Y(202)+Y(203)+Y(204)+Y(206)+Y(209)+Y(210)+Y(212)+Y(213)+Y(215)+Y(216)+Y(217)+Y(218)+Y(221)+Y(222)+Y(223)+Y(224)+Y(229)+Y(230)+Y(233)+Y(235)+Y(236)+Y(237)+Y(239)+Y(240)+Y(241)+Y(242)+Y(243)+Y(245

In [8]:
for spec in ODEs_cons:
    ODEs_cons = fortran_to_python(ODEs_cons, spec, cons = True)

e-
1
H2
1


### Parse when line is too long


In [9]:

def parse_line(parsed, cons = False):
    line = parsed[-1]

    if len(line) > 1000:
        for i in range(1000):
            n=i+1000
            if n < len(line):
                if line[n] == '+' or line[n] == '-':
                    break
            else:
                break
        parsed[-1] = line[0:n]

        if cons == True:
            parsed.append(line[0:4]+'+='+line[n:])
            
        if cons == False:
            parsed.append(line[0]+'+='+line[n:])

    else: 
        print('no')
    
    return 


In [10]:
# for line in ODEs['C']:

parsed = [ODEs['C'][0]]

while len(parsed[-1]) > 1000:
    parse_line(parsed)

In [11]:
for el in parsed:
    print(len(el))

1008
1006
1002
1017
1005
163


In [12]:
parsed

['F=0.+K[134]*X[2]*Y[14]+K[141]*Y[1]*Y[14]+K[147]*Y[10]*Y[455]+K[148]*Y[10]*Y[74]+K[149]*Y[10]*Y[82]+K[150]*Y[10]*Y[238]+K[151]*Y[10]*Y[154]+K[152]*Y[10]*Y[314]+K[153]*Y[10]*Y[280]+K[154]*Y[10]*Y[373]+K[155]*Y[10]*Y[274]+K[156]*Y[10]*Y[402]+K[157]*Y[10]*Y[397]+K[158]*Y[10]*Y[304]+K[159]*Y[10]*Y[244]+K[160]*Y[10]*Y[17]+K[161]*Y[10]*Y[152]+K[162]*Y[10]*Y[149]+K[163]*Y[10]*Y[157]+K[164]*Y[10]*Y[178]+K[165]*Y[10]*Y[158]+K[166]*Y[10]*Y[170]+K[167]*Y[10]*Y[207]+K[168]*Y[10]*Y[320]+K[169]*Y[10]*Y[232]+K[170]*Y[10]*Y[14]+K[171]*Y[10]*Y[186]+K[172]*Y[10]*Y[276]+K[173]*Y[10]*Y[316]+K[174]*Y[10]*Y[89]+K[175]*Y[10]*Y[127]+K[176]*Y[10]*Y[225]+K[177]*Y[10]*Y[270]+K[178]*Y[10]*Y[77]+K[179]*Y[10]*Y[325]+K[180]*Y[10]*Y[205]+K[181]*Y[10]*Y[256]+K[182]*Y[10]*Y[49]+K[185]*Y[10]*Y[35]+K[186]*Y[10]*Y[88]+K[187]*Y[10]*Y[227]+K[188]*Y[10]*Y[45]+K[189]*Y[10]*Y[331]+K[190]*Y[10]*Y[96]+K[191]*Y[10]*Y[106]+K[192]*Y[10]*Y[251]+K[193]*Y[10]*Y[258]+K[194]*Y[10]*Y[71]+K[195]*Y[10]*Y[282]+K[196]*Y[10]*Y[291]+K[197]*Y[

In [13]:
for spec in ODEs:
    print(spec)

    i = 0
    while i < len(ODEs[spec]):
        # print('i',i)

        if len(ODEs[spec][i]) > 1000:
            parsed = [ODEs[spec][i]]
            while len(parsed[-1]) > 1000:
                parse_line(parsed)

            # print('len parsed',len(parsed),'\n--\n')
            ODEs[spec] = ODEs[spec][0:i] + parsed + ODEs[spec][i+1:]

            i = i + len(parsed)
        
        else: i += 1

H
H+
H-
H2+
H3+
He
He+
HeH+
C-
C+
C
CH-
CH+
CH
CH2+
N+
CH2
N
CH3
NH
NH+
CH3+
O+
NH2+
CH4+
O-
NH2
O
CH4
OH-
OH
OH+
CH5+
NH3+
NH3
NH4+
H2O
H2O+
F
H3O+
F+
HF
HF+
H2F+
Na
Na+
C2-
C2+
Mg
Mg+
C2
C2H-
C2H+
C2H
CN
C2H2
C2H2+
CN-
CN+
HCN
C2H3
C2H3+
HCN+
HNC
N2
C2H4+
Si+
H2CN
H2NC+
N2+
Si
CO
CO+
C2H4
HCNH+
C2H5+
HCO
SiH+
HCO+
CH2NH
SiH
C2H5
HOC+
N2H+
CH2NH2+
SiH2
CH3CH3+
NO
H2CO
CH4N+
H2CO+
CH3CH3
SiH2+
NO+
P+
P
H3CO+
HNO+
SiH3+
HNO
CF+
C2H7+
SiH3
O2
SiH4
PH
S
H2NO+
CH3OH
SiH4+
O2+
S-
O2-
PH+
CH3OH+
S+
CH3OH2+
O2H
PH2+
HS
HS+
PH2
O2H+
SiH5+
H2O2
H2S+
H2S
PH3+
H3S+
Cl
Cl+
C3+
HCl
C3-
C3
HCl+
H2Cl+
C3H-
C3H+
C3H
C2N
C3H2+
C2N+
H2CCC
CNC+
C3H2
C2NH+
C3H3+
CH2CCH
CH2CCH+
SiC
CH2CCH2
SiC+
C2O
C3H4+
C2O+
CH2CN
CH3CCH
CH2CN+
C3H5+
HCSi
HCSi+
CH3CN+
HC2O+
CH3CN
C3H6+
SiN
CH3CNH+
NH2CN
CH3CHCH2
SiN+
SiCH2+
SiCH2
CNO
CH2CO+
OCN
OCN+
CH2CO
CP+
HNCO
HCNO+
HNCO+
HONC
SiCH3+
HCNO
CP
C3H7+
NH2CNH+
HNSi+
HOCN
HNSi
CH3CO+
HOCN+
SiCH3
HONC+
CS+
HCNOH+
N2O+
CS
SiCH4+
CO2+
H2OCN+
HNCOH+
SiNH2+
HCP
SiO+
CH3CHO
SiO
H

In [14]:
# parsed = ODEs_cons['e-']

# parse_line(parsed)

# print(len(parsed[-1]))
# while len(parsed[-1]) > 1000:
#     # print(parsed[-1])
#     parse_line(parsed)
        

### Write out in as python function

In [15]:
file = 'chem_ODEs.py'

## Y = number density of unconserved species
## X = number density of conserved species
## K = reaction rate
## ACCR = loss of H due to dust interaction, kind of rate

with open(file, 'w') as f:
    f.write('import numpy as np\n')
    f.write("\n'''\nIMPORTANT NOTE:\nThe content of this function is translated from Fortran 77. \nHence, indexing starts from 1. At position 0 there is a dummy species.\n \nWritten by Silke Maes, April 2023.\nOriginal Fortran code by rate10odecsT.pl on Sun, Jul 21 2019, \nusing ratefile: rate16_IP_10000K.rates, from http://udfa.ajmarkwick.net/index.php. \n'''")
    f.write('\ndef ODE(Y, X, K, ACCR, TOTAL):\n')
    f.write('\n\t## Conserved species\n')
    for key in ODEs_cons:
        f.write('\n\t## '+str(key)+'\n')
        for line in ODEs_cons[key]:
            f.write('\t'+line+'\n')
    f.write('\n\t## Non-conserved species\n')
    f.write('\tYDOT = np.zeros(len(Y))\n')
    for key in ODEs:
        f.write('\n\t## '+str(key)+'\n')
        for line in ODEs[key]:
            f.write('\t'+line+'\n')
    f.write('\n\treturn YDOT')